<a href="https://colab.research.google.com/github/ValentinaEmili/Ethnicity-recognition/blob/main/Dlib/68_landmarks/demo_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install validators
!pip install torchvision

In [6]:
import torch
from torchvision import transforms

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
# load the model pre-trained on ImageNet dataset
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
# modify the fully connected layer
resnet50.fc = torch.nn.Identity()
resnet50.eval().to(device)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [23]:
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import os
import math

def extract_and_save_features(file_path):
  image = ProbeImage(file_path=file_path, transform=transform)
  image_tensor, _ = image[0]
  image_tensor = image_tensor.to(device)
  with torch.no_grad():
    features = resnet50(image_tensor)
  return features.cpu().numpy().flatten()
  torch.cuda.empty_cache()  # Free up unused memory after each batch

class ProbeImage(Dataset):
  def __init__(self, file_path, transform=None):
    self.transform = transform
    self.file_path = file_path

  def __getitem__(self, index=0):
    image = Image.open(self.file_path).convert("RGB")
    image = self.transform(image).unsqueeze(0)
    file_name = os.path.basename(self.file_path)
    return image, file_name

  def __len__(self):
    return 1

In [16]:
import dlib
shape_predictor_path = "/content/drive/MyDrive/biometric_systems_project/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector() # HOG-based detector
predictor = dlib.shape_predictor(shape_predictor_path)

In [26]:
def shape_to_normal(shape):
  shape_normal = []
  for i in range(0, shape.num_parts):
    shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
  return shape_normal

def get_eyes_nose_dlib(shape):
  nose = shape[30][1]
  left_eye_x = (shape[36][1][0] + shape[39][1][0]) // 2
  left_eye_y = (shape[36][1][1] + shape[39][1][1]) // 2
  right_eyes_x = (shape[42][1][0] + shape[45][1][0]) // 2
  right_eyes_y = (shape[42][1][1] + shape[45][1][1]) // 2

  return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

def rotate_points(shape, rotation_matrix):
  points = []
  for i in range(0, 68):
    x,y = shape[i][1]
    points.append((x, y, 1))
  points = np.array(points)
  rotated_points = np.dot(rotation_matrix, points.T).T
  return np.array([(int(p[0]), int(p[1])) for p in rotated_points])

def get_eyes_in_image(image, points):
  height, width = image.shape[:2]
  top = points[17:27][np.argmin(points[17:27, 1])]
  bottom = points[29]
  min_x, min_y = np.min(points[36:48, 0]), np.min(points[36:48, 1])
  max_x, max_y = np.max(points[36:48, 0]), np.max(points[36:48, 1])
  min_x, min_y = max(0, min_x - 10), max(top[1], min_y - 10)
  max_x, max_y = min(width, max_x + 10), min(bottom[1], max_y + 10)
  image = image[min_y:max_y, min_x:max_x]
  return image

def get_nose_in_image(image, points):
  height, width = image.shape[:2]
  top = points[36:48][np.argmin(points[36:48, 1])]
  bottom = points[48:68][np.argmax(points[48:68, 1])]
  min_x, min_y = np.min(points[27:36, 0]), np.min(points[27:36, 1])
  max_x, max_y = np.max(points[27:36, 0]), np.max(points[27:36, 1])
  min_x, min_y = max(0, min_x - 5), max(top[1], min_y - 5)
  max_x, max_y = min(width, max_x + 5), min(bottom[1], max_y + 5)
  image = image[min_y:max_y, min_x:max_x]
  return image

def get_mouth_in_image(image, points):
  height, width = image.shape[:2]
  top = points[27:36][np.argmin(points[27:36, 1])]
  min_x, min_y = np.min(points[48:68, 0]), np.min(points[48:68, 1])
  max_x, max_y = np.max(points[48:68, 0]), np.max(points[48:68, 1])
  min_x, min_y = max(0, min_x - 10), max(top[1], min_y - 10)
  max_x, max_y = min(width, max_x + 10), min(height, max_y + 10)
  image = image[min_y:max_y, min_x:max_x]
  return image

def align_face(image, file_name):
  output_folder = "/content/drive/MyDrive/biometric_systems_project/trained_model"
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = detector(gray)
  if len(faces) > 0:
    for face in faces:
    # Extract face bounding box coordinates
      shape = predictor(gray, face)

      # Get facial landmarks for the detected face
      shape = shape_to_normal(shape)

      # Get the nose, left eye, and right eye positions
      nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
      delta_x = right_eye[0] - left_eye[0]
      delta_y = right_eye[1] - left_eye[1]
      # angle from radians to degrees
      angle = math.atan2(delta_y, delta_x) * (180 / math.pi)
      center_of_eyes = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
      rotation_matrix = cv2.getRotationMatrix2D(center_of_eyes, angle, scale=1.0)

      h, w = image.shape[:2]
      aligned_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)
      # catch features in the image
      rotated_points = rotate_points(shape, rotation_matrix)
      eyes_image = get_eyes_in_image(aligned_image, rotated_points)
      nose_image = get_nose_in_image(aligned_image, rotated_points)
      mouth_image = get_mouth_in_image(aligned_image, rotated_points)
      if eyes_image.size == 0:
        continue
      if nose_image.size == 0:
        continue
      if mouth_image.size == 0:
        continue
      else:
        eyes_image_path = os.path.join(output_folder, "eyes.jpg")
        mouth_image_path = os.path.join(output_folder, "mouth.jpg")
        nose_image_path = os.path.join(output_folder, "nose.jpg")
        cv2.imwrite(eyes_image_path, eyes_image)
        cv2.imwrite(mouth_image_path, mouth_image)
        cv2.imwrite(nose_image_path, nose_image)
        return eyes_image_path, mouth_image_path, nose_image_path

In [ ]:
import joblib
import os
import cv2

def main():
  file_path = "/content/drive/MyDrive/biometric_systems_project/trained_model/5_1_3_20170119154358954.jpg"
  folder_path = "/content/drive/MyDrive/biometric_systems_project/trained_model"
  characteristics = ["age", "ethnicity", "gender"]
  features = ["eyes", "mouth", "nose"]

  image = cv2.imread(file_path)
  eyes_image_path, nose_image_path, mouth_image_path = align_face(image, file_path)
  for feature, image_path in zip(features, [eyes_image_path, nose_image_path, mouth_image_path]):
    image_features = extract_and_save_features(image_path)
    for c in characteristics:
      feature_path = os.path.join(folder_path, c, f"svm_model_{feature}.pkl")
      model = joblib.load(feature_path)
      y_pred = model.predict([image_features])
      print(f"Prediction for {feature} - {c}: {y_pred[0]}")
if __name__ == '__main__':
  main()